In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [163]:
def getPricing(districtCode, schoolCode):
    DCode = fourDigitSchoolCode(districtCode)
    SCode = fourDigitSchoolCode(schoolCode)
    cost_url = 'https://www.cde.state.co.us/schoolview/financialtransparency/historical/'+str(DCode)+'/'+str(SCode)
    page = requests.get(cost_url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="maincontent")
    infoLinks = results.findAll("div", class_="infoLink")
    perStudentPricing = []
    for i in infoLinks:
        data = i.find("strong", class_="txt-large")
        if data:
            perStudentPricing.append(data.text.strip().replace('$','').replace(',',''))
    ## Only the first four entries are specific to the school. The others are state averages
    perStudentPricing = perStudentPricing[0:4]

    ## Now get the school budget
    totalSpendingDivs = results.findAll("div", class_="insetCta-inner theme-bg-0 theme-before-bg-0 theme-after-bg-0")
    totalSpending = []
    for i in totalSpendingDivs:
        data = i.find("div", class_="dataPoint-value").text.strip().replace('$','').replace(',','')
        totalSpending.append(data)
    ## Returns 2 arrays. The first is the per student for 2023, 2022, 2021, and 2020
    ## The second is the total school budget, for the same years
    return [*perStudentPricing, *totalSpending]

print(getPricing(130,1429))

['24263', '29415', '409067', '25025', '7477847', '17081964', '54094863', '2681486']


In [2]:
# def matchGrades(grade, code, gradeVal, allData):
#     if

# ## This creates the 
# def formatScores(dataframe, allData):
#     dataframe['3'] = dataframe.apply(lambda x: ('3', x['School Code'], x['Grade']) => 

In [3]:
## Not all school codes are 4 digits, so this takes any digit under four and appends 00s to the front as needed
def fourDigitSchoolCode(code):
    codeString = str(code)
    fill = '0000'
    if (len(codeString) == 4):
        return codeString
    return fill[0 : (4 - len(codeString))] + codeString

In [6]:
## Web scraper to get financial information for 2023. This includes the current school spending
## and the price per student
def getPricePerStudent(districtCode, schoolCode):
    # Note, historic data to 2019 exists at https://www.cde.state.co.us/schoolview/financialtransparency/historical/{district}/{school}
    baseUrl = 'https://www.cde.state.co.us/schoolview/financialtransparency/organizations/'
    codeDistrict = fourDigitSchoolCode(districtCode)
    codeSchool = fourDigitSchoolCode(schoolCode)
    page = requests.get(baseUrl + codeDistrict + '/' + codeSchool)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="maincontent")
    ## This is the span that includes the price per student value
    pricePerStudent = float(results.find("span", style="font-size: 2.5rem; font-weight: 700; line-height: 1;").text.replace('$','').replace(',',''))
    currentSchoolSpending = float(results.find("div", class_="dataPoint dataPoint_stacked dataPoint_large").find("div", class_="dataPoint-value").text.replace('$','').replace(',',''))
    ## To make sure this is working, just print the school codes
    print(schoolCode)
    return [pricePerStudent, currentSchoolSpending]

In [24]:
Results_2023 = pd.read_csv('2023 CMAS ELA and Math District and School Summary Achievement Results.csv', sep=",", header=0)
Results_2023 = Results_2023[(Results_2023['Level'] == 'SCHOOL') & (Results_2023['Content'] == 'Mathematics')]
## Will need to construct a new spreadsheet for each grade entry this way
# grades = ['3', '4', '5', '6', '7', '8']

print(Results_2023.query("`District Code`==10 and `School Code`==187 and Grade=='8'"))

       Level  District Code District Name  School Code  \
2691  SCHOOL             10    Mapleton 1          187   

                                    School Name      Content Grade  \
2691  Mapleton Expeditionary School of the Arts  Mathematics     8   

     Number of Total Records Number of Valid Scores Number of No Scores  ...  \
2691                      82                     80                   2  ...   

     Number Met Expectations Percent Met Expectations  \
2691                     - -                      - -   

     Number Exceeded Expectations Percent Exceeded Expectations  \
2691                          - -                           - -   

     Number Met or Exceeded Expectations 2023 2022  2019 Change\n2023-2022  \
2691                                 - -  - -  - -  15.7               - -   

                     Unnamed: 30  
2691  8th Congressional District  

[1 rows x 31 columns]


In [113]:
## THIS CODE BLOCK GENERATES THE SPEND DATA FOR EACH SCHOOL

## This is a web scraper that consolidates the historical spend data for Colorado schools at the school level
## This takes forever to run, so just use the csv.

## Not all school codes are 4 digits, so this takes any digit under four and appends 00s to the front as needed
def fourDigitSchoolCode(code):
    codeString = str(code)
    fill = '0000'
    if (len(codeString) == 4):
        return codeString
    return fill[0 : (4 - len(codeString))] + codeString

## Historical spend data per Student/School
def getPricing(districtCode, schoolCode):
    DCode = fourDigitSchoolCode(districtCode)
    SCode = fourDigitSchoolCode(schoolCode)
    cost_url = 'https://www.cde.state.co.us/schoolview/financialtransparency/historical/'+str(DCode)+'/'+str(SCode)
    page = requests.get(cost_url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="maincontent")
    infoLinks = results.findAll("div", class_="infoLink")
    perStudentPricing = []
    for i in infoLinks:
        data = i.find("strong", class_="txt-large")
        if data:
            perStudentPricing.append(data.text.strip().replace('$','').replace(',',''))
    ## Only the first four entries are specific to the school. The others are state averages
    perStudentPricing = perStudentPricing[0:4]

    ## Now get the school budget
    totalSpendingDivs = results.findAll("div", class_="insetCta-inner theme-bg-0 theme-before-bg-0 theme-after-bg-0")
    totalSpending = []
    for i in totalSpendingDivs:
        data = i.find("div", class_="dataPoint-value").text.strip().replace('$','').replace(',','')
        totalSpending.append(data)
    ## Returns 2 arrays. The first is the per student for 2023, 2022, 2021, and 2020
    ## The second is the total school budget, for the same years
    return [*perStudentPricing, *totalSpending]

Results_2023 = pd.read_csv('2023 CMAS ELA and Math District and School Summary Achievement Results.csv', sep=",", header=0)
Results_2023 = Results_2023[(Results_2023['Level'] == 'SCHOOL') & (Results_2023['Content'] == 'Mathematics') & (Results_2023['Grade'] == 'All Grades')]

df = pd.DataFrame()

df[['2023 Spend Per Student', '2022 Spend Per Student', '2021 Spend Per Student', '2020 Spend Per Student', 'Total Spending 2023', 'Total Spending 2022', 'Total Spending 2021', 'Total Spending 2020']]=Results_2023.apply(lambda x: getPricing(x['District Code'], x['School Code']),axis=1,result_type='expand')
df.to_csv('schools_budget_data.csv')

In [6]:
## This is the definitive collection of data. Since we only have price data to 2019, we will use that as our stopping point

## First, create a dataframe from the year's CSV data on performance
Results_2023 = pd.read_csv('2023 CMAS ELA and Math District and School Summary Achievement Results.csv', sep=",", header=0)

## Look at the school level performance for Mathematics, and take only the 'All Grades' Row. We will add in the grade-specific rows later
Results_2023 = Results_2023[(Results_2023['Level'] == 'SCHOOL') & (Results_2023['Content'] == 'Mathematics') & (Results_2023['Grade'] == 'All Grades')]

## Create empty columns for each existing grade. We want to rename them so we know what the data is
grades = ['3', '4', '5', '6', '7', '8']
for grade in grades:
    Results_2023['Grade ' + grade + ' % meets or exceeds expectations'] = 0

## Now, for each school, fill in the grade X meets expectations + exceeds expectations percentage
Results_2023 = Results_2023.apply(lambda x: getGradePerformance(x['District Code'], x['School Code'], Results_2023), axis=1)

## Now, we want to add the pricing data for 2020, 2021, 2022, and 2023


In [104]:
## THIS CODE BLOCK GENERATES THE DATA FOR ALL 

def remove_nulls(dataItem):
    # check the data type. If a string, remove the dollar sign
    # and any N/As
    if type(dataItem) is str:
        return(dataItem.replace('- -', '0').replace('N/A', '0'))
    return(dataItem)

## Function to add grade columns for each grade we find per school
def getGradePerformance(districtCode, schoolCode, df, grade):
    performance_2023 = 'N/A'
    performance_2022 = 'N/A'
    performance_2019 = 'N/A'
    mean_scale_score = 'N/A'
    result = df[(df['District Code'] == districtCode) & (df['School Code'] == schoolCode) & (df['Grade'] == grade)]
    if not result.empty:
        ## Note that these columns are not always populated, which is ridiculous
        performance_2023 = (result['2023'].apply(remove_nulls).astype('float')).array[0]
        performance_2022 = (result['2022'].apply(remove_nulls).astype('float')).array[0]
        performance_2019 = (result['2019'].apply(remove_nulls).astype('float')).array[0]
        mean_scale_score = (result['Mean Scale Score'].apply(remove_nulls).astype('float')).array[0]
    return [performance_2023, performance_2022, performance_2019, mean_scale_score]

    
Results_2023 = pd.read_csv('2023 CMAS ELA and Math District and School Summary Achievement Results.csv', sep=",", header=0)
Results_2023 = Results_2023[(Results_2023['Level'] == 'SCHOOL') & (Results_2023['Content'] == 'Mathematics')]

## Now, iterate through all the grades and add them to the master list
grades = ['3','4','5','6','7','8']
for grade in grades:
    Results_2023[['2023 Grade '+grade+' meets or exceeds expectations', '2022 Grade '+grade+' meets or exceeds expectations','2019 Grade '+grade+' meets or exceeds expectations', 'Grade '+grade+' Mean Scale Score']] = Results_2023.apply(lambda x: getGradePerformance(x['District Code'], x['School Code'], Results_2023, grade), axis=1,result_type='expand')
Finals_2023 = Results_2023[(Results_2023['Level'] == 'SCHOOL') & (Results_2023['Content'] == 'Mathematics') & (Results_2023['Grade'] == 'All Grades')]
# Finals_2023.to_csv('testing_this_out.csv')

## NOTE: Can merge based on column names: merged_df = DF2.merge(DF1, how = 'inner', on = ['date', 'hours'])
## TODO: Get 2022, 2021, 2019 average scores and performance by merging those spreadsheets
## Also: Add in the pricing data. Doing that now

In [118]:
## Now merge the finals together with the budget data
finals = pd.read_csv('testing_this_out.csv', sep=",", header=0)
budget = pd.read_csv('schools_budget_data.csv', sep=",", header=0)
result = pd.concat([finals, budget], axis=1, join='inner')

## NOTE: so full_data_2023 has lots of data, but we need test scores for each year from 2019. How do we get this?

RangeIndex(start=0, stop=1518, step=1)


In [160]:
## This code block creates the full csv with all data from 2019 - 2023 (omitting 2020)

def getGradePerformance(districtCode, schoolCode, grade, Results_2023, Results_2022, Results_2021, Results_2019):
    performance_2023 = 'N/A'
    performance_2022 = 'N/A'
    performance_2021 = 'N/A'
    performance_2019 = 'N/A'
    mean_scale_score_2023 = 'N/A'
    mean_scale_score_2022 = 'N/A'
    mean_scale_score_2021 = 'N/A'
    mean_scale_score_2019 = 'N/A'
    result_2023 = Results_2023[(Results_2023['District Code'] == districtCode) & (Results_2023['School Code'] == schoolCode) & (Results_2023['Grade'] == grade)]
    result_2022 = Results_2022[(Results_2022['District Code'] == districtCode) & (Results_2022['School Code'] == schoolCode) & (Results_2022['Grade'] == grade)]
    result_2021 = Results_2021[(Results_2021['District Code'] == int(districtCode)) & (Results_2021['School Code'] == int(schoolCode)) & (Results_2021['Grade'] == int(grade))]
    result_2019 = Results_2019[(Results_2019['District Code'] == districtCode) & (Results_2019['School Code'] == schoolCode) & (Results_2019['Grade'] == grade)]
    if not result_2023.empty:
        ## Note that these columns are not always populated, which is ridiculous
        performance_2023 = (result_2023['2023'].apply(remove_nulls).astype('float')).array[0]
        mean_scale_score_2023 = (result_2023['Mean Scale Score'].apply(remove_nulls).astype('float')).array[0]
        
    if not result_2022.empty:
        ## Note that these columns are not always populated, which is ridiculous
        performance_2022 = (result_2022['2022'].apply(remove_nulls).astype('float')).array[0]
        mean_scale_score_2022 = (result_2022['Mean Scale Score'].apply(remove_nulls).astype('float')).array[0]
        
    if not result_2021.empty:
        ## Note that these columns are not always populated, which is ridiculous
        performance_2021 = (result_2021['Percent Met or Exceeded Expectations'].apply(remove_nulls).astype('float')).array[0]
        mean_scale_score_2021 = (result_2021['Mean Scale Score'].apply(remove_nulls).astype('float')).array[0]
        
    if not result_2019.empty:
        ## Note that these columns are not always populated, which is ridiculous
        performance_2019 = (result_2019['Percent Met or Exceeded Expectations'].apply(remove_nulls).astype('float')).array[0]
        mean_scale_score_2019 = (result_2019['Mean Scale Score'].apply(remove_nulls).astype('float')).array[0]
        
    return [performance_2023, performance_2022, performance_2021, performance_2019, mean_scale_score_2023, mean_scale_score_2022, mean_scale_score_2021, mean_scale_score_2019]

## Now we need to get all the data from 2019 - 2022 and add it all in
## Need to extend the getGradePerformance function to do this
Results_2019 = pd.read_csv('2019 CMAS ELA MATH District and School Achievement Results.csv', sep=",", header=0)
Results_2019 = Results_2019[(Results_2019['Level'] == 'SCHOOL') & (Results_2019['Subject'] == 'Mathematics')]

Results_2021 = pd.read_csv('2021 CMAS ELA and Math District and School Summary Achievement Results - Required Tests.csv', sep=",", header=0)
Results_2021 = Results_2021[(Results_2021['Level'] == 'SCHOOL') & (Results_2021['Content'] == 'Mathematics')]

Results_2022 = pd.read_csv('2022 CMAS ELA and Math District and School Summary Achievement Results.csv', sep=",", header=0)
Results_2022 = Results_2022[(Results_2022['Level'] == 'SCHOOL') & (Results_2022['Content'] == 'Mathematics')]

Results_2023 = pd.read_csv('2023 CMAS ELA and Math District and School Summary Achievement Results.csv', sep=",", header=0)
Results_2023 = Results_2023[(Results_2023['Level'] == 'SCHOOL') & (Results_2023['Content'] == 'Mathematics')]

School_Budgets = pd.read_csv('schools_budget_data.csv', sep=",", header=0)
Results_2023.index = pd.RangeIndex(len(Results_2023.index))
Results_2023 = pd.concat([Results_2023, School_Budgets], axis=1, join='inner')

## Now, iterate through all the grades and add them to the master list
grades = ['3','4','5','6','7','8']
for grade in grades:
    Results_2023[[
        '2023 Grade '+grade+' meets or exceeds expectations',
        '2022 Grade '+grade+' meets or exceeds expectations',
        '2021 Grade '+grade+' meets or exceeds expectations',
        '2019 Grade '+grade+' meets or exceeds expectations',
        '2023 Grade '+grade+' Mean Scale Score',
        '2022 Grade '+grade+' Mean Scale Score',
        '2021 Grade '+grade+' Mean Scale Score',
        '2019 Grade '+grade+' Mean Scale Score']] = Results_2023.apply(lambda x: getGradePerformance(
            x['District Code'],
            x['School Code'],
            grade,
            Results_2023,
            Results_2022,
            Results_2021,
            Results_2019), axis=1,result_type='expand')
Finals_2023 = Results_2023[(Results_2023['Level'] == 'SCHOOL') & (Results_2023['Content'] == 'Mathematics') & (Results_2023['Grade'] == 'All Grades')]
Finals_2023.to_csv('All_The_Dates.csv')

In [174]:
Results_2023 = pd.read_csv('csvs/2023 CMAS ELA and Math District and School Summary Achievement Results.csv', sep=",", header=0)
Results_2023 = Results_2023[(Results_2023['Level'] == 'SCHOOL') & (Results_2023['Content'] == 'Mathematics')]
result_2023 = Results_2023[(Results_2023['District Code'] == 1040) & (Results_2023['School Code'] == 209) & (Results_2023['Grade'] == '8')]
validScores = result_2023['Number of Valid Scores'].values[0]
if validScores == '- -' or validScores == '< 16':
    print("Not gonna work")

Not gonna work


In [ ]:
## Old stuff below this

In [7]:
## This collects the 2023 price per student data for Colorado schools
Results_2023 = pd.read_csv('2023 CMAS ELA and Math District and School Summary Achievement Results.csv', sep=",", header=0)
## To remove duplicates, just take the codes and the All Grades value for Grade
Results_2023 = Results_2023[(Results_2023['Level'] == 'SCHOOL') & (Results_2023['Content'] == 'Mathematics') & (Results_2023['Grade'] == 'All Grades')]
## District Code, District Name, School Code, School Name, All Grades, Mean Scale Score, Standard Deviation, % Met Expectations
Results_2023 = Results_2023[['District Code', 'District Name', 'School Code', 'School Name', 'Grade']]
## Now add two new columns by getting the financial information
# Results_2023['Price Per Student'] = Results_2023.apply(lambda x: getPricePerStudent(x['District Code'], x['School Code']), axis=1)
Pricing_2023 = Results_2023.apply(lambda x: getPricePerStudent(x['District Code'], x['School Code']), axis=1)
Pricing_2023.to_csv('Pricing_2023.csv')

187
501
502
503
504
505
506
507
509
3590
3623
6915
6917
8823
14
57
59
210
301
1388
1480
1519
1878
1914
1937
2361
2410
2576
2578
2580
2582
2584
2918
3406
4000
4172
4187
5058
5418
5706
5814
6150
6342
6355
6376
6398
6802
6956
7155
7795
8211
8225
8275
8310
8361
8814
8842
9431
9444
9494
9682
20
186
1426
2308
4516
4536
5982
6534
7500
251
700
1013
1052
1796
2945
3900
4950
5615
5918
6395
6638
6687
6702
7131
7318
7351
7714
8032
8060
8130
8820
9230
9426
9801
562
774
8328
8332
1622
2876
3792
3931
4334
4465
5388
5834
7810
7812
7860
7952
8406
8798
9236
114
115
270
7626
7630
1514
1556
1652
2750
2752
9620
7837
16
18
102
141
188
242
243
348
442
714
1155
1273
1429
1510
1566
1568
1571
1572
1574
1614
1916
1970
2094
2292
2428
2897
3030
3648
3926
3988
4062
4078
4100
4189
4276
4280
4448
4975
5744
5934
6225
6820
6955
7102
7116
7158
7277
7476
7514
7559
7617
8380
8394
8848
8850
8887
9108
9200
9624
9802
1382
2219
2382
2804
2926
3472
4316
4447
5229
5233
5236
5572
5875
7518
7606
8064
9600
2136
2140
126
127
213
21